In [1]:
import warnings
warnings.filterwarnings('ignore')

import torchvision
import torch
from jetbot import Robot
    
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

In [2]:
model.load_state_dict(torch.load('go1.pth'))

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

In [4]:
robot = Robot()

In [5]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

# traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# display(image_widget)

In [7]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

In [8]:
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [9]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [10]:
# def display_xy(camera_image):
#     image = np.copy(camera_image)
#     x = x_slider.value
#     y = y_slider.value
#     x = int(x * 224 / 2 + 112)
#     y = int(y * 224 / 2 + 112)
#     image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
#     image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
#     image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
#     image = cv2.putText(image,"x="+str(x), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
#     jpeg_image = bgr8_to_jpeg(image)
#     return jpeg_image

image_widget2 = ipywidgets.Image()

# traitlets.dlink((camera, 'value'), (image_widget2, 'value'), transform=display_xy)

display(image_widget2)

Image(value=b'')

In [11]:
angle = 0.0
angle_last = 0.0

STEERING_GAIN = 0.75
STEERING_BIAS = 0.00


In [12]:
robot.left_motor.value = 0

In [13]:
def execute7(change):
    global angle, angle_last
#     image = change['new']
    image = camera.value
    preprocessed = preprocess(image)
    output = model(preprocessed).detach().cpu().numpy().flatten()
    # category_index = dataset.categories.index(category_widget.value)
    xx = output[0]
    yy = output[1]
    
    
    x = int(camera.width * (xx / 2.0 + 0.5))
    y = int(camera.height * (yy / 2.0 + 0.5))

    prediction = image.copy()
    prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
    prediction = cv2.line(prediction, (x,y), (112,224), (255,0,0), 3)
    prediction = cv2.putText(prediction,str(xx)+","+str(yy), (x-50,y+25), cv2.FONT_HERSHEY_PLAIN, 1, 2)
    prediction = cv2.putText(prediction,str(x)+","+str(y), (x-40,y+50), cv2.FONT_HERSHEY_PLAIN, 1, 2)       


    x_slider.value = x
    y_slider.value = y   
    
#     if xx < 0:
#         robot.left_motor.value = 0.1
#         robot.right_motor.value = 0.16
#     else:
#         robot.left_motor.value = 0.15
#         robot.right_motor.value = 0.1     
    

#     angle = np.arctan2(x, y)
#     pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
#     angle_last = angle
#     steering_slider.value = pid + steering_bias_slider.value
    
#     robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
#     robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)    

    prediction = cv2.putText(prediction,str(robot.left_motor.value)+","+str(robot.right_motor.value), (x-15,y+75), cv2.FONT_HERSHEY_PLAIN, 1, 2)               
    image_widget2.value = bgr8_to_jpeg(prediction)
    
    motor_r = 1 * 0.18 - .05 * xx
    motor_l = 1 * 0.18 + .05 * xx

    robot.left_motor.value = motor_l
    robot.right_motor.value = motor_r    

# execute({'new': camera.value})    

In [14]:
# execute6({'new': camera.value})    

In [15]:
display(image_widget2)

Image(value=b'')

In [16]:
camera.observe(execute7, names='value')

In [64]:
robot.stop()

In [63]:
camera.unobserve(execute7, names='value')
robot.stop()

ValueError: list.remove(x): x not in list

In [59]:
camera.unobserve(execute9, names='value')
robot.stop()

NameError: name 'execute9' is not defined

In [24]:
robot.stop()